## The Heat Equation in 1D

**PDE Form**

$$ \frac{\partial u}{\partial t} = \alpha \frac{\partial^2 u}{\partial x^2}, \quad 0 < x < L, t > 0 $$

Where:
- $u(x,t)$: Temperature at position $x$ and time $t$
- $\alpha$: Thermal diffusivity (a constant)
- $L$: Length of the rod

**Boundary Condition**
$$u(0,t) = 0, \quad u(L,t) = 0, \quad t \geqslant 0$$

**Initial Condition**
$$u(x, 0) = f(x) \quad 0\leqslant x \leqslant L$$

In [3]:
import numpy as np